In [2]:
import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Open the Parquet file to access metadata
parquet_file = pq.ParquetFile('train_series.parquet')

# df = pd.read_parquet('train_series.parquet')
event_df = pd.read_csv('train_events.csv')

In [6]:
def get_labels(series, events):
    output = np.zeros(len(series))
    events = events.copy()
    lower, upper = 0, 0
    for i, event in events.iterrows():
        if event["event"] == "onset":
            lower = event["step"]
        if event["event"] == "wakeup":
            upper = event["step"]
            output[int(lower) : int(upper)] = 1
    if lower > upper:
        output[int(lower):] = 1

    return output

def transform(all_series, all_events):
    all_events = all_events.dropna()
    transformed_df = all_series.copy()
    transformed_df['label'] = np.zeros(len(all_series))
    for series_id in all_series.series_id.unique():
        print(series_id)
        filtered_series = all_series[all_series['series_id'] == series_id]
        filtered_events = all_events[all_events['series_id'] == series_id]
        labels = get_labels(filtered_series, filtered_events)
        transformed_df.loc[transformed_df['series_id'] == series_id, 'label'] = labels
    return transformed_df

In [ ]:
n = 4
for i in range(n):
  df = pd.read_parquet(f'data/train_series_{i}.parquet')
  combined_df = transform(df, event_df)
  combined_df.to_parquet(f'data/combined_{i}.parquet')


